In [2]:
# Instala as bibliotecas da LangChain, OpenAI (para o cliente), NewsAPI e outras ferramentas
!pip install langgraph langchain-openai langchain-core pydantic python-dotenv beautifulsoup4 ipython nest_asyncio -q

# A biblioteca da OpenAI é usada para se conectar à API compatível do Ollama
!pip install openai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.5/152.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 20.3 MB/s eta 0:00:00


In [3]:
# Baixa e executa o script de instalação do Ollama
!curl -fsSL https://ollama.com/install.sh | sh

# Inicia o servidor Ollama em segundo plano e salva o log
# O nohup garante que o servidor continue rodando mesmo que a conexão do Colab se encerre
!nohup ollama serve &> ollama.serve.log &

# Aguarda alguns segundos para garantir que o servidor tenha tempo de iniciar
import time
time.sleep(5)

# Puxa o modelo qwen2:32b.
# AVISO: Este é um modelo grande (~19 GB). O download pode demorar
# e exigirá uma instância do Colab com GPU e RAM suficientes.
print("Iniciando o download do modelo qwen3:32b... Por favor, aguarde.")
!ollama pull qwen3:32b

print("\n--- Modelos Ollama disponíveis localmente: ---")
!ollama list

print("\n--- Verificando o status da GPU (NVIDIA): ---")
!nvidia-smi

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Iniciando o download do modelo qwen3:32b... Por favor, aguarde.


--- Modelos Ollama disponíveis localmente: ---
NAME         ID              SIZE     MODIFIED               
qwen3:32b    030ee887880f    20 GB    Less than a second ago    

--- Verificando o status da GPU (NVIDIA): ---
Mon Aug  4 11:59:10 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+-------------

In [4]:
import pandas as pd
import json
import os
from openai import OpenAI
from typing import List, Dict, Any
from tqdm.notebook import tqdm
from google.colab import drive
import nest_asyncio

# Permite o uso de loops de eventos aninhados, necessário em ambientes como o Colab
nest_asyncio.apply()

print("Bibliotecas importadas e ambiente configurado.")

Bibliotecas importadas e ambiente configurado.


In [5]:
class OllamaInference:
    """
    Classe otimizada para realizar inferência em lote usando um servidor Ollama local
    através de um cliente compatível com a API da OpenAI.
    """
    def __init__(self, model: str, system_prompt: str, structured_output_instructions: Dict[str, Any], api_key: str = "ollama"):
        """
        Inicializa a classe de inferência.

        Args:
            model (str): Nome do modelo a ser usado (ex: "qwen3:32b").
            system_prompt (str): Prompt de sistema comum para todas as chamadas.
            structured_output_instructions (Dict[str, Any]): Exemplo da estrutura JSON de saída.
            api_key (str): Chave de API para o serviço. Padrão "ollama" para uso local.
        """
        self.client = OpenAI(
            base_url="http://127.0.0.1:11434/v1",
            api_key=api_key
        )
        self.model = model
        # Combina o prompt do sistema com as instruções de formatação para maior precisão
        self.system_prompt = (
            f"{system_prompt}\n\n"
            f"Sua resposta DEVE ser um único objeto JSON com a seguinte estrutura: "
            f"{json.dumps(structured_output_instructions)}"
        )

    def _query_model(self, user_prompt: str) -> str:
        """Consulta um único modelo, com temperatura zero para saídas consistentes."""
        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": user_prompt},
        ]
        completion = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            temperature=0.0,
            response_format={"type": "json_object"} # Força a saída em formato JSON
        )
        return completion.choices[0].message.content

    def generate_outputs(self, dataset: pd.DataFrame) -> List[Dict[str, Any]]:
        """Processa um DataFrame e gera saídas para cada linha."""
        all_outputs = []
        for index, row in tqdm(dataset.iterrows(), total=len(dataset), desc=f"Processando com {self.model}"):
            user_prompt = f"Analise o seguinte texto:\n\n--- INÍCIO DO TEXTO ---\n{row['texto_ata']}\n--- FIM DO TEXTO ---"
            try:
                output = self._query_model(user_prompt)
                all_outputs.append({
                    "input": row.to_dict(),
                    "output": output
                })
            except Exception as e:
                print(f"Erro ao processar linha {index}: {e}")
                all_outputs.append({
                    "input": row.to_dict(),
                    "output": {"error": str(e)}
                })
        return all_outputs

In [13]:
# O modelo que será usado na inferência, conforme solicitado
MODEL_TO_USE = "qwen3:32b"

# Prompt de sistema claro e direto para a tarefa
system_prompt = (
    "Você é um especialista de análise de texto que são atas de reunião. Sua única tarefa é analisar o texto e indeficar o contexto de palavras e identificar se o texto da ata se trata de uma AUTORIZAÇÃO de IMPLANTAÇÃO e EXPLORAÇÃO DE CENTRAL GERADORAn"
    "REGRAS IMPORTANTES A SEGUIR:"
    "1. Sua resposta DEVE SER OBRIGATORIAMENTE um objeto JSON."
    "2. O objeto JSON DEVE OBRIGATORIAMENTE conter a chave 'autorizacao'."
    "3. Se o texto AUTORIZA a implantação, o valor da chave deve ser 'Autoriza'."
    "4. Se o texto NÃO autoriza, nega, trasfere ou é ambíguo, ou NÃO TEM RELAÇÃO com o tema, o valor da chave deve ser 'Não Autoriza'."
    "5. NÃO adicione explicações, comentários ou qualquer outro texto fora do objeto JSON."
    "6. As atas que autorizam SEMPRE tem os seguinte os dois termos na mesma ata: AUTORIZA, IMPLANTAR E EXPLORAR A CENTRAL GERADORA"
    "7. O valor chave de ser 'Não Autoriza' se o texto da ata contenham os seguintes termos: 'RESOLUÇÃO HOMOLOGATÓRIA'; 'ALTERAÇÃO'; 'ALTERA'; 'DECLARA DE UTILIDADE PÚBLICA'; 'TRANSFERE'; 'REVOGA'; 'ALTERA'; 'ALTERAR'"
  )

# Instruções de saída focadas apenas na resposta desejada, para máxima precisão
structured_output_instructions = {
    "autorizacao": "Descreva em uma única palavra: 'Autoriza' ou 'Não Autoriza'"
}

print(f"Tarefa configurada para o modelo: {MODEL_TO_USE}")

Tarefa configurada para o modelo: qwen3:32b


In [7]:
# Monta o Google Drive para acessar os arquivos
drive.mount('/content/drive')

# Caminho para o seu arquivo CSV
dataset_path = '/content/drive/MyDrive/MBABIGDATAIA/DATASET/DATASET-5YTD.csv'

print("Carregando dataset completo...")
full_dataset = pd.read_csv(dataset_path)

# Cria uma amostra aleatória de 50 registros para um teste rápido.
# (random_state garante que a amostra seja sempre a mesma em execuções diferentes)
sample_dataset = full_dataset.sample(n=372, random_state=42)

print(f"Dataset completo carregado com {len(full_dataset)} registros.")
print(f"Amostra de teste criada com {len(sample_dataset)} registros.")

display(sample_dataset.head())

Mounted at /content/drive
Carregando dataset completo...
Dataset completo carregado com 10264 registros.
Amostra de teste criada com 372 registros.


,nome_arquivo,texto_ata
2507,dsp20211902.txt,AGÊNCIA NACIONAL DE ENERGIA ELÉTRICA – ANEEL \...
5501,dsp20241467.txt,AGÊNCIA NACIONAL DE ENERGIA ELÉTRICA – ANEEL \...
932,rea202212921ti.txt,AGÊNCIA NACIONAL DE ENERGIA ELÉTRICA — ANEEL \...
1190,dsp2022361.txt,AGÊNCIA NACIONAL DE ENERGIA ELÉTRICA – ANEEL \...
2619,rea202211784ti.txt,\n \n \nAGÊNCIA NACIONAL DE ENERGIA ELÉTRICA ...


In [14]:
# 1. Instancia a classe com todas as configurações
inference_runner = OllamaInference(
    model=MODEL_TO_USE,
    system_prompt=system_prompt,
    structured_output_instructions=structured_output_instructions
)

# 2. Gera as saídas para o conjunto de dados de amostra
raw_results = inference_runner.generate_outputs(sample_dataset)

# 3. Processa os resultados brutos para um formato limpo
final_results = []
for item in raw_results:
    input_data = item['input']
    output_str = item['output']

    # Valor padrão para erros e casos não autorizados
    autorizacao_value = 'Não Autoriza'

    try:
        # Tenta carregar a string de saída como JSON
        output_json = json.loads(output_str)

        # Verifica se a chave 'autorizacao' existe e se o valor é 'Autoriza'
        if 'autorizacao' in output_json and output_json['autorizacao'] == 'Autoriza':
            autorizacao_value = 'Autoriza'
        else:
             # Se a chave não existe, o valor não é 'Autoriza', ou há um erro no JSON retornado pelo modelo
             autorizacao_value = 'Não Autoriza'

    except (json.JSONDecodeError, TypeError) as e:
        # Este erro acontece se a saída do modelo não for um JSON válido.
        print(f"ERRO DE PARSING ou TIPO para '{input_data.get('nome_arquivo')}': {e}. Saída do modelo: {output_str}")
        autorizacao_value = 'Não Autoriza' # Classifica como Não Autoriza em caso de erro

    final_results.append({
        'nome_arquivo': input_data.get('nome_arquivo'),
        'texto_ata': input_data.get('texto_ata'),
        'autorizacao_modelo': autorizacao_value
    })

# 4. Cria o DataFrame final
results_df = pd.DataFrame(final_results)

# 5. Salva e exibe os resultados
output_path = '/content/drive/MyDrive/MBABIGDATAIA/DATASET/DATASETAMOSTRA-5YTD-AUTORIZADO.csv'
results_df.to_csv(output_path, index=False)

print(f"\nResultados salvos com sucesso em: {output_path}")
print(f"Total de registros processados: {len(results_df)}")
display(results_df.head())

Processando com qwen3:32b:   0%|          | 0/372 [00:00<?, ?it/s]


Resultados salvos com sucesso em: /content/drive/MyDrive/MBABIGDATAIA/DATASET/DATASETAMOSTRA-5YTD-AUTORIZADO.csv
Total de registros processados: 372


,nome_arquivo,texto_ata,autorizacao_modelo
0,dsp20211902.txt,AGÊNCIA NACIONAL DE ENERGIA ELÉTRICA – ANEEL \...,Não Autoriza
1,dsp20241467.txt,AGÊNCIA NACIONAL DE ENERGIA ELÉTRICA – ANEEL \...,Não Autoriza
2,rea202212921ti.txt,AGÊNCIA NACIONAL DE ENERGIA ELÉTRICA — ANEEL \...,Autoriza
3,dsp2022361.txt,AGÊNCIA NACIONAL DE ENERGIA ELÉTRICA – ANEEL \...,Não Autoriza
4,rea202211784ti.txt,\n \n \nAGÊNCIA NACIONAL DE ENERGIA ELÉTRICA ...,Não Autoriza


In [15]:
# Filtra o DataFrame para incluir apenas as linhas onde 'autorizacao_modelo' é 'Autoriza'
authorized_results_df = results_df[results_df['autorizacao_modelo'] == 'Autoriza']

# Exibe o DataFrame filtrado
print("\n--- Registros Autorizados ---")
display(authorized_results_df)


--- Registros Autorizados ---


,nome_arquivo,texto_ata,autorizacao_modelo
2,rea202212921ti.txt,AGÊNCIA NACIONAL DE ENERGIA ELÉTRICA — ANEEL \...,Autoriza
13,rea202110388ti.txt,AGÊNCIA NACIONAL DE ENERGIA ELÉTRICA – ANEEL \...,Autoriza
19,rea202212511ti.txt,\nAGÊNCIA NACIONAL DE ENERGIA ELÉTRICA – ANEE...,Autoriza
21,rea202110076ti.txt,\nAGÊNCIA NACIONAL DE ENERGIA ELÉTRICA – ANEE...,Autoriza
22,rea202314413ti.txt,\nAGÊNCIA NACIONAL DE ENERGIA ELÉTRICA – ANEE...,Autoriza
...,...,...,...
359,rea202313809ti.txt,\nAGÊNCIA NACIONAL DE ENERGIA ELÉTRICA – ANEE...,Autoriza
361,rea202313951ti.txt,AGÊNCIA NACIONAL DE ENERGIA ELÉTRICA – ANEEL \...,Autoriza
362,rea202314472ti.txt,AGÊNCIA NACIONAL DE ENERGIA ELÉTRICA – ANEEL \...,Autoriza
366,rea202211589ti.txt,\nAGÊNCIA NACIONAL DE ENERGIA ELÉTRICA – ANEE...,Autoriza


In [23]:
# Caminho completo para salvar o arquivo CSV filtrado
output_filtered_path = '/content/drive/MyDrive/MBABIGDATAIA/DATASET/DATASET-AUTORIZADO.csv'

# Salva o DataFrame filtrado em um novo arquivo CSV
authorized_results_df.to_csv(output_filtered_path, index=False)

print(f"Dataset filtrado salvo com sucesso em: {output_filtered_path}")

Dataset filtrado salvo com sucesso em: /content/drive/MyDrive/MBABIGDATAIA/DATASET/DATASET-AUTORIZADO.csv


In [28]:
# Célula 2: Carregar o novo dataset

# Monta o Google Drive, se ainda não estiver montado
drive.mount('/content/drive', force_remount=True)

# --- ATENÇÃO: ATUALIZE O CAMINHO ABAIXO ---
# Coloque o caminho exato para o seu novo arquivo CSV com os casos autorizados.
autorizados_dataset_path = '/content/drive/MyDrive/MBABIGDATAIA/DATASET/DATASET-AUTORIZADO.csv' # Exemplo de nome de arquivo

print(f"Carregando dataset de autorizados de: {autorizados_dataset_path}")
try:
    autorizados_df = pd.read_csv(autorizados_dataset_path)
    print(f"Dataset carregado com sucesso! Contém {len(autorizados_df)} registros para extração de dados.")
    display(autorizados_df.head())
except FileNotFoundError:
    print("\nERRO: Arquivo não encontrado! Por favor, verifique o caminho e o nome do arquivo em 'autorizados_dataset_path'.")

Mounted at /content/drive
Carregando dataset de autorizados de: /content/drive/MyDrive/MBABIGDATAIA/DATASET/DATASET-AUTORIZADO.csv
Dataset carregado com sucesso! Contém 95 registros para extração de dados.


,nome_arquivo,texto_ata,autorizacao_modelo
0,rea202212921ti.txt,AGÊNCIA NACIONAL DE ENERGIA ELÉTRICA — ANEEL \...,Autoriza
1,rea202110388ti.txt,AGÊNCIA NACIONAL DE ENERGIA ELÉTRICA – ANEEL \...,Autoriza
2,rea202212511ti.txt,\nAGÊNCIA NACIONAL DE ENERGIA ELÉTRICA – ANEE...,Autoriza
3,rea202110076ti.txt,\nAGÊNCIA NACIONAL DE ENERGIA ELÉTRICA – ANEE...,Autoriza
4,rea202314413ti.txt,\nAGÊNCIA NACIONAL DE ENERGIA ELÉTRICA – ANEE...,Autoriza


In [29]:
# Célula 3: Configuração da Tarefa de Extração

MODEL_TO_USE = "qwen3:32b" # Voltando para o modelo de alta performance

# Novo prompt de sistema, focado em extração de dados
system_prompt = (
    "Você é um assistente de IA especialista em extrair informações estruturadas de documentos regulatórios da ANEEL. "
    "Sua tarefa é ler o texto de uma ata de reunião, que já foi previamente identificada como uma AUTORIZAÇÃO, e extrair os dados específicos solicitados. "
    "Se uma informação específica não for encontrada de forma clara no texto, retorne o valor 'ERRO' para a chave correspondente."
)

# Nova estrutura de saída JSON para os dados detalhados
structured_output_instructions = {
    "nome_empresa": "O nome completo da empresa, consórcio ou pessoa física que recebeu a autorização.",
    "tipo_geracao": "O tipo da fonte de energia (ex: Fotovoltaica, Eólica, Hídrica, Termelétrica).",
    "nome_central_geradora": "O nome oficial da Central Geradora (ex: UFV Sol do Sertão, EOL Ventos do Sul).",
    "municipio": "O município onde a central será instalada.",
    "estado": "A sigla do estado (UF) onde a central será instalada.",
    "unidades_geradoras": "A quantidade total de unidades geradoras (ex: número de turbinas ou inversores). Extraia apenas o número.",
    "potencia_geracao_kw": "A potência de cada unidade geradora (kW). Extraia apenas o número.",
    "potencia_instalada_kw": "A potência total instalada em kilowatts (kW). Extraia apenas o número.",
    "data_resolucao": "A data em que a resolução autorizativa foi emitida, no formato AAAA-MM-DD.",
    "prazo_limite_meses": "O prazo, em meses, para a implantação da central. Extraia apenas o número.",
}

print("Novo prompt e estrutura de extração definidos com sucesso!")

Novo prompt e estrutura de extração definidos com sucesso!


In [32]:
# Célula 4: Executar Extração e Montar DataFrame Final

# 1. Instancia a classe com a nova configuração de extração
extraction_runner = OllamaInference(
    model=MODEL_TO_USE,
    system_prompt=system_prompt,
    structured_output_instructions=structured_output_instructions
)

# 2. Gera as saídas para o dataset de autorizados
# Se o dataset for muito grande, considere processar uma amostra primeiro: autorizados_df.sample(n=10)
raw_extracted_results = extraction_runner.generate_outputs(autorizados_df)

# 3. Processa os resultados JSON e monta a lista final
final_extracted_data = []
for item in raw_extracted_results:
    input_data = item['input']
    output_str = item['output']

    extracted_data = {}
    try:
        parsed_json = json.loads(output_str)
        # Se o modelo retornar um erro interno, capture-o
        if 'error' in parsed_json:
             print(f"INFO: Modelo retornou erro para o arquivo '{input_data.get('nome_arquivo')}': {parsed_json['error']}")
             # Preenche com Nulos se houver erro
             for key in structured_output_instructions.keys():
                 extracted_data[key] = None
        else:
            extracted_data = parsed_json

    except (json.JSONDecodeError, TypeError):
        print(f"ERRO DE PARSING: A saída para '{input_data.get('nome_arquivo')}' não é um JSON válido: {output_str}")
        # Preenche com Nulos se o JSON for inválido
        for key in structured_output_instructions.keys():
            extracted_data[key] = None

    # Combina os dados originais (como nome_arquivo) com os dados extraídos
    final_row = {**input_data, **extracted_data}
    final_extracted_data.append(final_row)

# 4. Cria o DataFrame final
extracted_df = pd.DataFrame(final_extracted_data)

# 5. Salva e exibe os resultados
extracted_output_path = '/content/drive/MyDrive/MBABIGDATAIA/DATASET/DATASET-FINAL.csv'
extracted_df.to_csv(extracted_output_path, index=False)

print(f"\nExtração de dados concluída! Resultados salvos em: {extracted_output_path}")
print("Amostra do resultado final:")

# Exibe as primeiras linhas do resultado. O .T transpõe para melhor visualização se houver muitas colunas.
display(extracted_df.head().T)

Processando com qwen3:32b:   0%|          | 0/95 [00:00<?, ?it/s]


Extração de dados concluída! Resultados salvos em: /content/drive/MyDrive/MBABIGDATAIA/DATASET/DATASET-FINAL.csv
Amostra do resultado final:


,0,1,2,3,4
nome_arquivo,rea202212921ti.txt,rea202110388ti.txt,rea202212511ti.txt,rea202110076ti.txt,rea202314413ti.txt
texto_ata,AGÊNCIA NACIONAL DE ENERGIA ELÉTRICA — ANEEL \...,AGÊNCIA NACIONAL DE ENERGIA ELÉTRICA – ANEEL \...,\nAGÊNCIA NACIONAL DE ENERGIA ELÉTRICA – ANEE...,\nAGÊNCIA NACIONAL DE ENERGIA ELÉTRICA – ANEE...,\nAGÊNCIA NACIONAL DE ENERGIA ELÉTRICA – ANEE...
autorizacao_modelo,Autoriza,Autoriza,Autoriza,Autoriza,Autoriza
nome_empresa,Cei Solar Empreendimentos Energéticos S/A,EDP Renováveis Brasil S.A.,Fótons de São Valentim Energias Renováveis S.A.,Usina Alto Alegre S.A. – Açúcar e Álcool,Omega Desenvolvimento de Energia 17 S.A.
tipo_geracao,Fotovoltaica,Eólica,Fotovoltaica,Termelétrica,Fotovoltaica
nome_central_geradora,UFV Barreiro XVIII,EOL Serra da Borborema V,UFV Fótons de São Valentim 01,UTE UJU Bio,UFV Kuara 6 XII
municipio,Capitão Enéas,Esperança,Lajes,Colorado,Icapuí
estado,MG,PB,RN,PR,CE
unidades_geradoras,16,6,12,1,7
potencia_geracao_kw,3000,6200,4167,50000,4200
